In [ ]:
### MZ_pyfesom_overview


This is an example for evaluation of FESOM2-REcoM output. 
Adapted from py_recom/GlobalAssesment/MASTER_Script.ipynb

Please install:
- pyfesom2
- skillmetrics
- cartopy

link to the official pyfesom2 documentation:
https://pyfesom2.readthedocs.io/en/latest/index.html



In [ ]:
%matplotlib inline
%autosave 5

In [ ]:
# necessary modules -------------------------------------------------------------------------------------
import pyfesom2 as pf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cartopy.crs as ccrs

# own py_recom modules ----------------------------------------------------------------------
!jupyter nbconvert --to=python load_interp_WOA_python3.ipynb
!jupyter nbconvert --to=python plot_Taylor_normalized.ipynb

#from load_interp_WOA_python3 import WOAdata
#from plot_Taylor_normalized import plt_Taylor_norm

# Settings

In [ ]:
# run specification -------------------------------------------------------------------------------------
runid      =  'fesom'
layerwise    = False
resultpath = '/work/ollie/mozeisin/results/f2r1.2/mo5'
meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

# period of analysis 
first_year = 1955
last_year  = 1957
    
years = [first_year, last_year]

# specification of analysis ------------------------------------------------------------------------------------
depth = 0
layerwise = False
mapproj = 'pc'
#Map projection. Options are Mercator (merc), Plate Carree (pc),
#North Polar Stereo (np), South Polar Stereo (sp),  Robinson (rob)
#Robinson projection is quite time consuming.

# export of analysis ------------------------------------------------------------------------------------
savepath = ''
savefig = False
htmlname     =  'TEST_' + runid +'.html'
htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'

# initialization file specifications -----------------------------------------------------------
ncfileDIN                = '/work/ollie/projects/MarESys/evaluation/woa13_all_n00_01.nc'
ncfileTemp               = '/work/ollie/projects/MarESys/evaluation/woa18_decav_t00_01.nc'


# Load FESOM mesh

### pf.load_mesh(meshpath)

In [ ]:
# mesh initialization 
mesh = pf.load_mesh(meshpath)

In [ ]:
# check mesh setup nodal and area

meshdiag=resultpath+'/'+runid+'.mesh.diag.nc'
#!ncdump -h $meshdiag

### pf.get_meshdiag()

In [ ]:
diag = pf.get_meshdiag(mesh,meshdiag=meshdiag, runid=runid)

In [ ]:
# depth of layers

mesh_depths = diag['Z'].values
print(mesh_depths)

In [ ]:
# Nodal area

nod_area = diag.rename_dims({"nl": "nz1", "nod_n": "nod2"}).nod_area
nod_area.load()

print(np.shape(nod_area[:,:]), type(nod_area))
    

### pf.plot()

In [ ]:
# whole globe
pf.plot(mesh,np.array(nod_area[0,:]),units='Surface nodal area')

### pf.tplot()

In [ ]:
# plot nodal area and mesh structure together
pf.tplot(mesh, np.array(nod_area[0,:]), ptype='tri', box=[-30, 30, 60, 82], mapproj='np',lw=0.5,units='nodal area')
pf.tplot(mesh, np.array(nod_area[0,:]), ptype='tri', box=[-30, 30, -30, 30], mapproj='merc',lw=0.5,units='nodal area')

# Physical fields

In [ ]:
NCfesom = resultpath + '/DIC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Dissolved inorganic Carbon {0}-{1} at {2} m'.format(years[0],years[1],depth)
units = 'DIC [mmol  m$^{-3}$]'

### pf.get_data()

In [ ]:
DIC = pf.get_data(resultpath, "DIC", years, mesh, runid=runid, how="mean", compute=True, depth=depth)

### output shape

In [ ]:
print(np.shape(DIC))

### pf.plot()

In [ ]:
fig = pf.plot(mesh, DIC, mapproj = mapproj, units=units, titles=label)

### switch projection

#### mapproj: str
Map projection. 

Options are Mercator (merc), Plate Carree (pc), North Polar Stereo (np), South Polar Stereo (sp),  Robinson (rob)

Help: pf.plot?

In [ ]:
fig = pf.plot(mesh, DIC, mapproj = 'rob', units=units, titles=label)

# Biology

# Diatom carbon

In [ ]:
NCfesom = resultpath + '/DiaC.fesom.1957.nc'
!ncdump -h $NCfesom

### pf.get_data()

In [ ]:
label='Diatom intracell. Carbon {0}-{1} at {2} m'.format(years[0],years[1],depth)
units = 'intracellular C [mmol C m$^{-3}$]'

DiaC = pf.get_data(resultpath, "DiaC", years, mesh, runid=runid, how="mean", compute=True, depth=depth)

fig = pf.plot(mesh, DiaC, mapproj = mapproj, units=units, titles=label)

### compare runs

In [ ]:
# define various runs at once
resultpath_base = '/work/ollie/mozeisin/results/f2r1.2'
path_list = '/mo4', '/mo5', '/mo6', '/mo7'
resultpath_list = [resultpath_base +i for i in path_list]

# corresponding runid of FESOM runs
idlist = ('fesom', 'fesom', 'fesom', 'fesom') 

In [ ]:
# load various runs at once
DiaC = [0]*len(idlist)

for i in range(len(idlist)):
    print(resultpath_list[i])
    DiaC[i] = pf.get_data(resultpath_list[i], "DiaC", years, mesh, runid=idlist[i], how="mean", compute=True, depth=depth)
    DiaC[i] = DiaC[i] * 12.01

In [ ]:
# plot various runs at once
units = 'Diatom intracell. Carbon [mg C m$^{{-3}}$]\n{0}-{1} at {2} m'.format(years[0],years[1],depth)

figlabel = ('mo4', 'mo5', 'mo6', 'mo7')

fig = pf.plot(mesh, [data for data in DiaC], mapproj = mapproj, units=units, cmap = 'viridis',
              rowscol = (2,2),
              figsize = (15,10),
              titles=[title for title in figlabel])

In [ ]:
## differences
fig = pf.plot(mesh, [data-DiaC[0] for data in DiaC], mapproj = mapproj, units='MODEL - CTRL\n'+units, 
              rowscol = (2,2),  
              figsize = (15,10),
              #levels = (-2.5e-2,2.5e-2, 30), 
              cmap = 'RdBu',
              titles=[title for title in figlabel])

# Dissolved inorganic Nitrogen

In [ ]:
NCfesom = resultpath + '/DIN.fesom.1957.nc'
!ncdump -h $NCfesom

### pf.get_data(silent = True)

In [ ]:
DIN = pf.get_data(resultpath, "DIN", years, mesh, runid=runid, how="mean", compute=True, depth=depth)

In [ ]:
label='Dissolved inorganic Nitrogen {0}-{1} at {2} m'.format(years[0],years[1],depth)
units = 'DIN [mmol m$^{-3}$]'

fig = pf.plot(mesh, DIN, mapproj = mapproj, units=units, titles=label)

# Comparison scripts

### WOA: temperature

In [ ]:
!jupyter nbconvert --to=python Temp_WOA_Comparison_python3.ipynb

from Temp_WOA_Comparison_python3 import TEMPcomp
test = TEMPcomp(runid,resultpath,savepath,meshpath,ncfileTemp,first_year,last_year,mapproj=mapproj,savefig=savefig)


# Transect

In [ ]:
# define Atlantic transect
lon_start = -25
lat_start = -50
lon_end = -25
lat_end = 65
npoints   = 50

### pf.transect_get_lonlat()

In [ ]:
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

### pf.plot_transect_map()

In [ ]:
pf.plot_transect_map(lonlat, mesh)

### load 3d data

In [ ]:
#load 3D data
DiaC_3d = pf.get_data(resultpath, "DiaC", years, mesh, how="mean")

DiaC_3d = DiaC_3d * 12.01

print(np.shape(DiaC_3d))

### pf.get_transect()

In [ ]:
dist, transect_data = pf.get_transect(DiaC_3d,mesh,lonlat)

### pf.plot_xyz()

In [ ]:
units = 'Diatom [mg C m$^{{-3}}$]'
title = 'Atlantic Transect, \nmean of {0}-{1} at {2} m'.format(years[0],years[1],depth)

pf.plot_xyz(mesh, 
            transect_data,
            dist,
            maxdepth=500,
            ncols=2,
            xlabel='Distance',
            label=units,
            title=title,
            levels=np.linspace(0,40,41),#cmap=cm.hot_r,
            )

# Interpolation to 1x1 grid

In [ ]:
# interpolation prep

lat = np.arange(-90,90,1)
#print(lats)
lon = np.arange(-180,180,1)

lons, lats = np.meshgrid(lon, lat)

print('Original variable shape on FESOM mesh:')
print(np.shape(DIC))

### pf.fesom2regular()

In [ ]:
#pf.fesom2regular?

In [ ]:
DIC_interp = pf.fesom2regular(
        data = DIC,
        mesh = mesh,
        lons = lons, 
        lats = lats)

In [ ]:
print('Interpolated variable shape on regular grid:')
print(np.shape(DIC_interp))

### now normal plotting is possible

In [ ]:
fig = plt.figure(figsize=(10,10))
ax1 = plt.subplot(projection =  ccrs.Robinson())

m1 = plt.pcolormesh(lon, lat, DIC_interp, 
    transform = ccrs.PlateCarree(),
    cmap=plt.cm.viridis)

ax1.coastlines(resolution='110m', color='black', linewidth=1)

cbar = fig.colorbar(m1,orientation = 'horizontal',fraction=0.1, pad=0.1) 
cbar.set_label('Dissolved Inorganic Carbon [mmol  m$^{-3}$]', fontsize=20)